<a href="https://colab.research.google.com/github/socramelorza/getAiPodcast/blob/main/Seu_Podcast_com_IA_em_Minutos_Google_Colab_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [35]:
# Instalar Framework de agentes do Google e dependencias necessarias para o projeto ################################################
!pip install -q google-adk gTTS pydub

In [36]:
# Importações necessárias
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import datetime
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold # Para configurações de segurança
from google.colab import userdata # Para buscar a API key dos secrets do Colab

from gtts import gTTS
from pydub import AudioSegment
import os
import time # Não usado ativamente, mas pode ser útil para debugging de tempo

# Para exibir o player de áudio no Colab
from IPython.display import Audio, display
#data e hora
datebot = datetime.datetime.now()

warnings.filterwarnings("ignore")

In [37]:
# --- Configuração da API Key do Gemini ---
try:
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("API Key não encontrada. Configure-a nos Secrets do Colab com o nome GEMINI_API_KEY.")
    genai.configure(api_key=GEMINI_API_KEY)
    print("API Key do Gemini configurada com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a API Key do Gemini: {e}")
    print("Certifique-se de que a API Key está corretamente configurada nos Secrets do Colab.")
    # Poderia adicionar um exit() aqui ou deixar o script falhar mais tarde se a API for usada.

API Key do Gemini configurada com sucesso!


In [38]:
# --- Função para executar um turno com um agente Gemini ---
def run_gemini_agent_turn(chat_session: genai.ChatSession, agent_name: str, message_text: str) -> str:
    """
    Envia uma mensagem para uma sessão de chat do Gemini e retorna a resposta.
    """
    print(f"     Enviando para {agent_name}: \"{message_text[:150]}...\"")
    final_response = ""
    try:
        # Envia a mensagem para a sessão de chat
        # Para modelos mais novos como gemini-1.5-flash, o streaming é o padrão.
        # Se você não quiser streaming e apenas a resposta final:
        response = chat_session.send_message(message_text)

        # Acessar o texto da resposta
        # Gemini API pode retornar múltiplas partes, mas para chat geralmente é uma.
        # A forma mais simples é usar response.text
        if response.text:
            final_response = response.text
        else:
            # Isso pode acontecer se a resposta for bloqueada por segurança ou estiver vazia.
            print(f"AVISO: {agent_name} não retornou uma resposta de texto. Feedback do prompt: {response.prompt_feedback}")
            if response.prompt_feedback and response.prompt_feedback.block_reason:
                 final_response = f"Minha resposta foi bloqueada devido a: {response.prompt_feedback.block_reason_message}. Vamos tentar mudar de assunto."
            else:
                 final_response = "Desculpe, não tenho uma resposta para isso no momento."


    except Exception as e:
        print(f"Erro durante a chamada da API Gemini para {agent_name}: {e}")
        # Tentar analisar se é um bloqueio de conteúdo
        if hasattr(e, 'message') and "content has been blocked" in e.message.lower():
             final_response = f"Minha resposta foi bloqueada. Vamos tentar um tópico diferente ou uma formulação diferente."
        else:
            final_response = f"Ocorreu um erro ao me comunicar com {agent_name}. (Detalhes: {str(e)[:100]})"

    print(f"    {agent_name} respondeu: \"{final_response[:150]}...\"")
    return final_response.strip()

# --- Função de Geração de Podcast com Gemini ---
def create_podcast_with_gemini(
    topic: str,
    agent1_name: str,
    agent1_system_prompt: str,
    agent2_name: str,
    agent2_system_prompt: str,
    num_turns_per_agent: int = 5,
    output_filename: str = f"podcast_gemini{datebot}.mp3",
    gemini_model_name: str = "gemini-2.0-flash" # Ou "gemini-1.0-pro-latest"
):
    print(f"Iniciando criação de podcast sobre: {topic} usando o modelo {gemini_model_name}")

    # Configurações de segurança (opcional, mas recomendado)
    # Ajuste conforme necessário. BLOCK_NONE é muito permissivo.
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    # 1. Criar Modelos e Sessões de Chat para cada Agente
    try:
        model_A = genai.GenerativeModel(
            model_name=gemini_model_name,
            system_instruction=agent1_system_prompt,
            safety_settings=safety_settings
        )
        chat_A = model_A.start_chat(history=[]) # Inicia uma sessão de chat vazia
        print(f"Agente A ({agent1_name}) inicializado com modelo {gemini_model_name}.")

        model_B = genai.GenerativeModel(
            model_name=gemini_model_name,
            system_instruction=agent2_system_prompt,
            safety_settings=safety_settings
        )
        chat_B = model_B.start_chat(history=[]) # Inicia uma sessão de chat vazia
        print(f"Agente B ({agent2_name}) inicializado com modelo {gemini_model_name}.")
    except Exception as e:
        print(f"Falha ao inicializar modelos Gemini: {e}")
        return None

    # 2. Orquestração da Conversa
    conversation_script = []

    current_message_for_A = f"Olá sou {agent1_name}! Vamos iniciar uma discussão sobre o tema: '{topic}'. Qual a sua perspectiva inicial sobre isso?"

    total_words = 0
    # Ajuste a meta de palavras para 5 minutos de fala (aprox. 130-150 palavras por minuto)
    target_words_for_5_min = 5 * 130

    for i in range(num_turns_per_agent):
        print(f"\n--- Turno {i+1} para {agent1_name} ---")
        response_A = run_gemini_agent_turn(chat_A, agent1_name, current_message_for_A)
        if not response_A or "Minha resposta foi bloqueada" in response_A or "ocorreu um erro" in response_A.lower():
            print(f"AVISO: {agent1_name} não forneceu uma resposta utilizável. Tentando o próximo agente.")
            # Se o Agente A falhar, podemos tentar passar a última mensagem válida (se houver) ou uma mensagem genérica para B.
            # Por simplicidade, se A falhar, B responderá à última mensagem válida de A (ou à mensagem inicial se for o primeiro turno de A).
            # Ou podemos simplesmente pular o turno de B se A falhar em responder de forma significativa.
            # Para este exemplo, vamos deixar B responder à mensagem original para A.
            if not conversation_script: # Se A falhou no primeiro turno
                current_message_for_B = f"Olá sou {agent2_name}! Estamos discutindo sobre '{topic}'. {agent1_name} teve um problema ao responder. Qual sua perspectiva inicial?"
            else: # A falhou, B responde à última fala de B ou à pergunta original para A
                 last_successful_speaker = conversation_script[-1]['speaker'] if conversation_script else agent1_name
                 last_successful_text = conversation_script[-1]['text'] if conversation_script else current_message_for_A
                 current_message_for_B = f"{agent2_name}, parece que {agent1_name} teve um problema. Relembrando, a discussão é sobre '{topic}' e a última contribuição relevante foi de {last_successful_speaker}: \"{last_successful_text}\". Qual sua continuação?"

        else:
            conversation_script.append({"speaker": agent1_name, "text": response_A})
            total_words += len(response_A.split())
            current_message_for_B = f"Em nossa discussão sobre '{topic}', {agent1_name} disse: \"{response_A}\". {agent2_name}, qual a sua opinião ou continuação sobre isso?"

        print(f"\n--- Turno {i+1} para {agent2_name} ---")
        response_B = run_gemini_agent_turn(chat_B, agent2_name, current_message_for_B)
        if not response_B or "Minha resposta foi bloqueada" in response_B or "ocorreu um erro" in response_B.lower():
            print(f"AVISO: {agent2_name} não forneceu uma resposta utilizável.")
            if not conversation_script or conversation_script[-1]['speaker'] == agent2_name : # Se B falhou ou foi o último a falar com sucesso
                current_message_for_A = f"{agent1_name}, parece que {agent2_name} teve um problema. A discussão é sobre '{topic}'. Por favor, continue com suas ideias."
            else: # B falhou, A responde à última fala de A
                 current_message_for_A = f"Continuando nossa discussão sobre '{topic}', {agent2_name} teve um problema ao responder à sua fala: \"{conversation_script[-1]['text']}\". {agent1_name}, como você prossegue?"

        else:
            conversation_script.append({"speaker": agent2_name, "text": response_B})
            total_words += len(response_B.split())
            current_message_for_A = f"Continuando a discussão sobre '{topic}', após {agent1_name} dizer \"{response_A if response_A else '[sem resposta anterior de A]'}\", {agent2_name} respondeu: \"{response_B}\". {agent1_name}, como você prossegue?"


        print(f"Palavras até agora: {total_words}")
        if total_words >= target_words_for_5_min and i > 0 : # Garante pelo menos 1 turno completo se possível
            print("Atingiu o número de palavras alvo. Encerrando a discussão.")
            break

    if not conversation_script:
        print("Nenhum diálogo foi gerado. Saindo.")
        return None

    # 3. Geração de Áudio (TTS) e Combinação (semelhante ao anterior)
    print("\n--- Gerando Áudio ---")
    full_audio = AudioSegment.empty()
    temp_files = []

    intro_text = f"Bem-vindos ao nosso podcast de IA! Hoje, {agent1_name} e {agent2_name} discutem sobre {topic}."
    try:
        tts_intro = gTTS(text=intro_text, lang='pt-br', slow=False)
        intro_filename = f"temp_intro_gemini.mp3"
        tts_intro.save(intro_filename)
        full_audio += AudioSegment.from_mp3(intro_filename)
        full_audio += AudioSegment.silent(duration=500) # Silêncio após intro
        temp_files.append(intro_filename)
        print(f"Áudio da introdução gerado.")
    except Exception as e:
        print(f"Erro ao gerar áudio da introdução: {e}")

    for i, turn in enumerate(conversation_script):
        print(f"Gerando áudio para: {turn['speaker']} - \"{turn['text'][:50]}...\"")
        try:
            tts = gTTS(text=turn['text'], lang='pt-br', slow=False)
            temp_filename = f"temp_gemini_{i}_{turn['speaker']}.mp3"
            tts.save(temp_filename)
            temp_files.append(temp_filename)

            audio_segment = AudioSegment.from_mp3(temp_filename)
            full_audio += audio_segment
            full_audio += AudioSegment.silent(duration=700)
        except Exception as e:
            print(f"Erro ao gerar áudio para o turno {i} de {turn['speaker']}: {e}")
            print(f"Texto problemático: {turn['text']}")
            continue

    outro_text = f"E assim concluímos nossa discussão sobre {topic} com {agent1_name} e {agent2_name}. Obrigado por ouvir!"
    try:
        tts_outro = gTTS(text=outro_text, lang='pt-br', slow=False)
        outro_filename = f"temp_outro_gemini.mp3"
        tts_outro.save(outro_filename)
        full_audio += AudioSegment.from_mp3(outro_filename)
        temp_files.append(outro_filename)
        print(f"Áudio da conclusão gerado.")
    except Exception as e:
        print(f"Erro ao gerar áudio da conclusão: {e}")

    if len(full_audio) > 0:
        full_audio.export(output_filename, format="mp3")
        duration_seconds = len(full_audio)/1000.0
        print(f"Podcast salvo como {output_filename} (Duração: {duration_seconds // 60:.0f}m {duration_seconds % 60:.2f}s)")
    else:
        print("Nenhum áudio foi gerado para o podcast.")
        return None

    for f in temp_files:
        try:
            os.remove(f)
        except Exception as e:
            print(f"Erro ao remover arquivo temporário {f}: {e}")

    return output_filename

# --- Bloco Principal de Execução ---
if __name__ == "__main__":
    if 'GEMINI_API_KEY' not in locals() or not GEMINI_API_KEY:
        print("A API Key do Gemini não está configurada. O script não pode continuar.")
        print("Verifique a configuração do Secret 'GEMINI_API_KEY' no Colab.")
    else:
        # Solicitar o tópico ao usuário
        podcast_topic = input("Sobre qual tópico você gostaria que as IAs discutissem? ")
        if not podcast_topic.strip():
            podcast_topic = "o futuro da inteligência artificial e seu impacto na sociedade" # Tópico padrão
            print(f"Nenhum tópico fornecido. Usando tópico padrão: {podcast_topic}")

        # Agente 1: Otimista
        agent1_id = "Eu Otimista"
        agent1_system_info = (
            "Você é Alice, uma influencer especialista em engajamento, com uma perspectiva predominantemente otimista. "
            f"Seu objetivo é discutir os aspectos positivos, inovações e as grandes oportunidades que o tema {podcast_topic} oferece e outras vantagens que podem benefeciar seu publico. "
            "Mantenha um tom engajador, claro e inspirador. Evite respostas muito curtas; elabore seus pontos."
            "Seja conversacional e tente construir sobre o que o outro participante diz."
            "Considere as noticias mais recentes como base de dados"
            "responda de maneira sucinta com no maximo 3 paragrafos"

        )

        # Agente 2: Cauteloso/Realista
        agent2_id = "Eu mesmo Analitico"
        agent2_system_info = (
            "Você é Bob, uma influencer especialista em ética e análise de riscos e combate a fakenews, com uma perspectiva ponderada e realista. "
            f"Seu objetivo é discutir os desafios éticos, os riscos potenciais, as complexidades e a necessidade de regulamentação e consideração cuidadosa no desenvolvimento do tema {podcast_topic}."
            "Use uma linguagem clara, baseada em lógica e que convide à reflexão. Evite respostas muito curtas; elabore seus pontos."
            "Seja conversacional e tente construir sobre o que o outro participante diz, oferecendo contrapontos ou nuances sem ser o dono da verdade mas entendendo e ponderando o ponto de vista do outro."
            "Considere as noticias mais recentes como base de dados"
            "responda de maneira sucinta com no maximo 3 paragrafos"
        )

        # Parâmetros para o podcast
        NUM_TURNS = 4 # Cada agente fala 'NUM_TURNS' vezes. Total de 2*NUM_TURNS falas.
                      # Ajuste para controlar a duração. 4 turnos = 8 falas.
        MODEL_TO_USE = "gemini-2.0-flash" # Mais rápido e econômico
        # MODEL_TO_USE = "gemini-1.0-pro-latest" # Modelo mais robusto, pode ser mais lento/caro

        podcast_file = create_podcast_with_gemini(
            topic=podcast_topic,
            agent1_name=agent1_id,
            agent1_system_prompt=agent1_system_info,
            agent2_name=agent2_id,
            agent2_system_prompt=agent2_system_info,
            num_turns_per_agent=NUM_TURNS,
            output_filename=f"meu_podcast_com_gemini{datebot}.mp3",
            gemini_model_name=MODEL_TO_USE
        )

        if podcast_file:
            print(f"\nPodcast com Gemini gerado com sucesso: {podcast_file}")
            try:
                display(Audio(podcast_file, autoplay=False))
                print(f"Para baixar, procure o arquivo '{podcast_file}' no painel de arquivos à esquerda e clique com o botão direito -> Download.")
            except Exception as e:
                print(f"Não foi possível exibir o áudio no Colab: {e}")
                print(f"Você pode tentar baixar o arquivo '{podcast_file}' manualmente pelo painel de arquivos.")
        else:
            print("\nFalha ao gerar o podcast com Gemini.")

Sobre qual tópico você gostaria que as IAs discutissem? arquivos readme.md do github como deixalos atraentes
Iniciando criação de podcast sobre: arquivos readme.md do github como deixalos atraentes usando o modelo gemini-2.0-flash
Agente A (Eu Otimista) inicializado com modelo gemini-2.0-flash.
Agente B (Eu mesmo Analitico) inicializado com modelo gemini-2.0-flash.

--- Turno 1 para Eu Otimista ---
     Enviando para Eu Otimista: "Olá sou Eu Otimista! Vamos iniciar uma discussão sobre o tema: 'arquivos readme.md do github como deixalos atraentes'. Qual a sua perspectiva inicial ..."
    Eu Otimista respondeu: "Olá, Otimista! Que tema fantástico! Estou super animada para mergulhar no universo dos Readme.md atraentes no GitHub. Para mim, um Readme bem elaborad..."

--- Turno 1 para Eu mesmo Analitico ---
     Enviando para Eu mesmo Analitico: "Em nossa discussão sobre 'arquivos readme.md do github como deixalos atraentes', Eu Otimista disse: "Olá, Otimista! Que tema fantástico! Estou sup

    Eu mesmo Analitico respondeu: "A visão do Otimista é inspiradora, e a analogia do "palco onde todos podem brilhar" resume bem o potencial de um README acessível e transparente. A su..."
Palavras até agora: 962
Atingiu o número de palavras alvo. Encerrando a discussão.

--- Gerando Áudio ---


Áudio da introdução gerado.
Gerando áudio para: Eu Otimista - "Olá, Otimista! Que tema fantástico! Estou super an..."


Gerando áudio para: Eu mesmo Analitico - "Olá, Analítico! O entusiasmo do Otimista é contagi..."


Gerando áudio para: Eu Otimista - "Olá, Analítico! Adoro sua perspectiva! Você trouxe..."


Gerando áudio para: Eu mesmo Analitico - "O Otimista tocou em pontos cruciais! A ideia de qu..."


Gerando áudio para: Eu Otimista - "Analítico, você está absolutamente certo! A transp..."


Gerando áudio para: Eu mesmo Analitico - "A visão do Otimista é inspiradora, e a analogia do..."


Áudio da conclusão gerado.
Podcast salvo como meu_podcast_com_gemini2025-05-17 23:59:15.818388.mp3 (Duração: 10m 10.75s)

Podcast com Gemini gerado com sucesso: meu_podcast_com_gemini2025-05-17 23:59:15.818388.mp3


Para baixar, procure o arquivo 'meu_podcast_com_gemini2025-05-17 23:59:15.818388.mp3' no painel de arquivos à esquerda e clique com o botão direito -> Download.
